In [1]:
import pandas as pd
import random
import uuid
import json

web_events = pd.read_csv('web_events.csv')
web_events.head(2)

/var/folders/m0/28qj953n24g5gqv9dl1hcf6w0000gn/T/ipykernel_25257/3495403620.py:6: DtypeWarning: Columns (43) have mixed types. Specify dtype option on import or set low_memory=False.
  web_events = pd.read_csv('web_events.csv')


,APP_ID,PLATFORM,ETL_TSTAMP,COLLECTOR_TSTAMP,DVCE_CREATED_TSTAMP,EVENT,EVENT_ID,TXN_ID,NAME_TRACKER,V_TRACKER,...,EVENT_NAME,EVENT_FORMAT,EVENT_VERSION,EVENT_FINGERPRINT,TRUE_TSTAMP,LOAD_TSTAMP,CONTEXTS_COM_SNOWPLOWANALYTICS_SNOWPLOW_UA_PARSER_CONTEXT_1,CONTEXTS_COM_SNOWPLOWANALYTICS_SNOWPLOW_WEB_PAGE_1,CONTEXTS_COM_IAB_SNOWPLOW_SPIDERS_AND_ROBOTS_1,CONTEXTS_NL_BASJES_YAUAA_CONTEXT_1
0,website,web,2022-08-19 23:04:17.974,2022-08-19 23:04:17.482,2022-08-19 23:04:15.214,page_view,a365ec7d-23d7-47f1-9afb-1025175b802c,NaN,biz1,js-3.5.0,...,page_view,jsonschema,1-0-0,5ffc35492251cbfe07ea491602037cb7,NaN,NaN,"""[{'deviceFamily':'iPhone','osFamily':'iOS','o...","""[{'id':'81655a36-1201-4cba-95d7-7702937888ba'}]""","""[{'category':'BROWSER','primaryImpact':'NONE'...","""[{'agentClass':'Browser','agentName':'Safari'..."
1,website,web,2022-08-19 23:04:17.980,2022-08-19 23:04:17.482,2022-08-19 23:04:15.238,struct,d4ffa8b6-b93a-4f03-9f14-5e9f2457e2b6,NaN,biz1,js-3.5.0,...,event,jsonschema,1-0-0,646f12ed17fecbc2bb744f3a5509c705,NaN,NaN,"""[{'deviceFamily':'iPhone','osFamily':'iOS','o...","""[{'id':'81655a36-1201-4cba-95d7-7702937888ba'}]""","""[{'category':'BROWSER','primaryImpact':'NONE'...","""[{'agentClass':'Browser','agentName':'Safari'..."


In [2]:
mobile_events = pd.read_csv('mobile_events.csv')
mobile_events.head(2)

,APP_ID,PLATFORM,ETL_TSTAMP,COLLECTOR_TSTAMP,DVCE_CREATED_TSTAMP,EVENT,EVENT_ID,TXN_ID,NAME_TRACKER,V_TRACKER,...,UNSTRUCT_EVENT_ORG_IETF_HTTP_COOKIE_1,UNSTRUCT_EVENT_COM_MAILGUN_RECIPIENT_UNSUBSCRIBED_1,UNSTRUCT_EVENT_COM_MAILGUN_MESSAGE_CLICKED_1,UNSTRUCT_EVENT_COM_MAILGUN_MESSAGE_OPENED_1,UNSTRUCT_EVENT_COM_MAILGUN_MESSAGE_DELIVERED_1,UNSTRUCT_EVENT_COM_MAILGUN_MESSAGE_COMPLAINED_1,CONTEXTS_COM_SNOWPLOWANALYTICS_MOBILE_DEEP_LINK_1,CONTEXTS_COM_SNOWPLOWANALYTICS_MOBILE_APPLICATION_LIFECYCLE_1,UNSTRUCT_EVENT_COM_SNOWPLOWANALYTICS_MOBILE_DEEP_LINK_RECEIVED_1,UNSTRUCT_EVENT_COM_SNOWPLOWANALYTICS_MOBILE_MESSAGE_NOTIFICATION_1
0,com.snowplowanalytics.SnowplowSwiftSPMDemo,mob,2022-09-15 07:33:46.465,2022-09-15 07:33:45.609,2022-09-15 07:33:45.500,unstruct,c307e014-e08f-418b-930c-a91b3de5f560,NaN,DemoAppNamespace,ios-4.0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[\n {\n ""isVisible"": true\n }\n]",NaN,NaN
1,com.snowplowanalytics.SnowplowSwiftSPMDemo,mob,2022-09-15 07:33:43.521,2022-09-15 07:33:42.604,2022-09-15 07:33:41.378,unstruct,e0b77e5f-faff-426d-96d5-34889ab2cf79,NaN,DemoAppNamespace,ios-4.0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[\n {\n ""isVisible"": true\n }\n]",NaN,NaN


In [3]:
user_columns = [
    'V_COLLECTOR',
    'V_ETL',
    'USER_ID',
    'USER_IPADDRESS',
    'USER_FINGERPRINT',
    'NETWORK_USERID',
    'GEO_COUNTRY',
    'GEO_REGION',
    'GEO_CITY',
    'GEO_ZIPCODE',
    'GEO_LATITUDE',
    'GEO_LONGITUDE',
    'GEO_REGION_NAME',
    'IP_ISP',
    'IP_ORGANIZATION',
    'IP_DOMAIN',
    'IP_NETSPEED',
    'OS_NAME',
    'OS_FAMILY',
    'OS_MANUFACTURER',
    'OS_TIMEZONE',
    'DVCE_TYPE',
    'DVCE_ISMOBILE',
    'DVCE_SCREENWIDTH',
    'DVCE_SCREENHEIGHT',
    'DOC_CHARSET',
    'DOC_WIDTH',
    'DOC_HEIGHT',
    'TR_CURRENCY',
    'TI_CURRENCY',
    'TI_PRICE_BASE',
    'BASE_CURRENCY',
    'GEO_TIMEZONE',
]
users = web_events.drop_duplicates(user_columns).reset_index(drop=True)[user_columns]

In [4]:
screen_views = mobile_events.loc[~mobile_events['UNSTRUCT_EVENT_COM_SNOWPLOWANALYTICS_MOBILE_SCREEN_VIEW_1'].isna()]
screen_view_users = screen_views.groupby('NETWORK_USERID').size()
screen_view_users = screen_view_users.loc[(screen_views.groupby('NETWORK_USERID').size() > 1)].index
screen_views = screen_views.loc[screen_views['NETWORK_USERID'].isin(screen_view_users)]

In [7]:
generated_df = pd.DataFrame()

for _, user in users.sample(100).iterrows():
    user_id = str(uuid.uuid4())
    previous_session_id = None

    subtract_days = random.randint(10, 360)

    for session_i in range(random.randint(1, 3)):
        screen_view_user = random.sample(list(screen_view_users), k=1)[0]
        user_screen_views = screen_views.loc[screen_views['NETWORK_USERID'] == screen_view_user]
        user_screen_views = user_screen_views.head(random.randint(1, len(user_screen_views))).copy()
        user_screen_views['EVENT_ID'] = [str(uuid.uuid4()) for _ in range(len(user_screen_views))]

        # change timestamps
        for ts_col in [ 'ETL_TSTAMP', 'COLLECTOR_TSTAMP', 'DVCE_CREATED_TSTAMP', 'DVCE_SENT_TSTAMP', 'DERIVED_TSTAMP' ]:
            user_screen_views[ts_col] = (pd.to_datetime(user_screen_views[ts_col]) - pd.Timedelta(days=subtract_days / (session_i + 1))).astype(str)

        # update session ids
        session_id = str(uuid.uuid4())
        session = [
            {
                "firstEventId": user_screen_views.iloc[0]['EVENT_ID'],
                "firstEventTimestamp": user_screen_views.iloc[0]['DVCE_CREATED_TSTAMP'],
                "previousSessionId": previous_session_id,
                "sessionId": session_id,
                "sessionIndex": session_i + 1,
                "storageMechanism": "LOCAL_STORAGE",
                "userId": user_id
            }
        ]
        previous_session_id = session_id
        user_screen_views['CONTEXTS_COM_SNOWPLOWANALYTICS_SNOWPLOW_CLIENT_SESSION_1'] = json.dumps(session)

        parsed_screen_entity = user_screen_views['UNSTRUCT_EVENT_COM_SNOWPLOWANALYTICS_MOBILE_SCREEN_VIEW_1'].apply(
            lambda item: json.loads(item)
        )
        screen_id_mappings = {
            i: str(uuid.uuid4())
            for i in parsed_screen_entity.apply(lambda r: r['id'])
        }

        # regenerate screen view id
        user_screen_views['UNSTRUCT_EVENT_COM_SNOWPLOWANALYTICS_MOBILE_SCREEN_VIEW_1'] = parsed_screen_entity.apply(
            lambda item: json.dumps({**item, **{
                'id': screen_id_mappings[item['id']],
                'previousId': screen_id_mappings.get(item['previousId']),
            }})
        )

        user_screen_views['CONTEXTS_COM_SNOWPLOWANALYTICS_MOBILE_SCREEN_1'] = user_screen_views['CONTEXTS_COM_SNOWPLOWANALYTICS_MOBILE_SCREEN_1'].apply(
            lambda item: json.dumps([{**json.loads(item)[0], **{
                'id': screen_id_mappings[json.loads(item)[0]['id']]
            }}])
        )

        # set user columns
        for c in users.columns:
            user_screen_views[c] = user[c]

        generated_df = pd.concat([
            generated_df,
            user_screen_views
        ])


In [8]:
final_columns = [
    'APP_ID',
    'PLATFORM',
    'ETL_TSTAMP',
    'COLLECTOR_TSTAMP',
    'DVCE_CREATED_TSTAMP',
    'EVENT',
    'EVENT_ID',
    'TXN_ID',
    'NAME_TRACKER',
    'V_TRACKER',
    'V_COLLECTOR',
    'V_ETL',
    'USER_ID',
    'USER_IPADDRESS',
    'USER_FINGERPRINT',
    'DOMAIN_USERID',
    'DOMAIN_SESSIONIDX',
    'NETWORK_USERID',
    'GEO_COUNTRY',
    'GEO_REGION',
    'GEO_CITY',
    'GEO_ZIPCODE',
    'GEO_LATITUDE',
    'GEO_LONGITUDE',
    'GEO_REGION_NAME',
    'IP_ISP',
    'IP_ORGANIZATION',
    'IP_DOMAIN',
    'IP_NETSPEED',
    'PAGE_URL',
    'PAGE_TITLE',
    'PAGE_REFERRER',
    'PAGE_URLSCHEME',
    'PAGE_URLHOST',
    'PAGE_URLPORT',
    'PAGE_URLPATH',
    'PAGE_URLQUERY',
    'PAGE_URLFRAGMENT',
    'REFR_URLSCHEME',
    'REFR_URLHOST',
    'REFR_URLPORT',
    'REFR_URLPATH',
    'REFR_URLQUERY',
    'REFR_URLFRAGMENT',
    'REFR_MEDIUM',
    'REFR_SOURCE',
    'REFR_TERM',
    'MKT_MEDIUM',
    'MKT_SOURCE',
    'MKT_TERM',
    'MKT_CONTENT',
    'MKT_CAMPAIGN',
    'SE_CATEGORY',
    'SE_ACTION',
    'SE_LABEL',
    'SE_PROPERTY',
    'SE_VALUE',
    'TR_ORDERID',
    'TR_AFFILIATION',
    'TR_TOTAL',
    'TR_TAX',
    'TR_SHIPPING',
    'TR_CITY',
    'TR_STATE',
    'TR_COUNTRY',
    'TI_ORDERID',
    'TI_SKU',
    'TI_NAME',
    'TI_CATEGORY',
    'TI_PRICE',
    'TI_QUANTITY',
    'PP_XOFFSET_MIN',
    'PP_XOFFSET_MAX',
    'PP_YOFFSET_MIN',
    'PP_YOFFSET_MAX',
    'USERAGENT',
    'BR_NAME',
    'BR_FAMILY',
    'BR_VERSION',
    'BR_TYPE',
    'BR_RENDERENGINE',
    'BR_LANG',
    'BR_FEATURES_PDF',
    'BR_FEATURES_FLASH',
    'BR_FEATURES_JAVA',
    'BR_FEATURES_DIRECTOR',
    'BR_FEATURES_QUICKTIME',
    'BR_FEATURES_REALPLAYER',
    'BR_FEATURES_WINDOWSMEDIA',
    'BR_FEATURES_GEARS',
    'BR_FEATURES_SILVERLIGHT',
    'BR_COOKIES',
    'BR_COLORDEPTH',
    'BR_VIEWWIDTH',
    'BR_VIEWHEIGHT',
    'OS_NAME',
    'OS_FAMILY',
    'OS_MANUFACTURER',
    'OS_TIMEZONE',
    'DVCE_TYPE',
    'DVCE_ISMOBILE',
    'DVCE_SCREENWIDTH',
    'DVCE_SCREENHEIGHT',
    'DOC_CHARSET',
    'DOC_WIDTH',
    'DOC_HEIGHT',
    'TR_CURRENCY',
    'TR_TOTAL_BASE',
    'TR_TAX_BASE',
    'TR_SHIPPING_BASE',
    'TI_CURRENCY',
    'TI_PRICE_BASE',
    'BASE_CURRENCY',
    'GEO_TIMEZONE',
    'MKT_CLICKID',
    'MKT_NETWORK',
    'ETL_TAGS',
    'DVCE_SENT_TSTAMP',
    'REFR_DOMAIN_USERID',
    'REFR_DVCE_TSTAMP',
    'DOMAIN_SESSIONID',
    'DERIVED_TSTAMP',
    'EVENT_VENDOR',
    'EVENT_NAME',
    'EVENT_FORMAT',
    'EVENT_VERSION',
    'EVENT_FINGERPRINT',
    'TRUE_TSTAMP',
    'LOAD_TSTAMP',
    'CONTEXTS_COM_SNOWPLOWANALYTICS_MOBILE_SCREEN_1',
    'CONTEXTS_COM_SNOWPLOWANALYTICS_SNOWPLOW_CLIENT_SESSION_1',
    'CONTEXTS_COM_SNOWPLOWANALYTICS_SNOWPLOW_GEOLOCATION_CONTEXT_1',
    'CONTEXTS_COM_SNOWPLOWANALYTICS_SNOWPLOW_MOBILE_CONTEXT_1',
    'CONTEXTS_COM_SNOWPLOWANALYTICS_MOBILE_APPLICATION_1',
    'UNSTRUCT_EVENT_COM_SNOWPLOWANALYTICS_MOBILE_SCREEN_VIEW_1',
]

In [9]:
len(final_columns), len(generated_df.columns)

(135, 187)

In [10]:
generated_df[final_columns].to_csv('generated.csv', index=False)